In [ ]:
#Make sure that you have all these libaries available to run the code successfully
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from math import sqrt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import math
from datetime import datetime
from fbprophet import Prophet
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
from numpy import mean
from numpy import std

In [ ]:
#loading data 
mydateparser = lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
dataset = pd.read_csv('data10_25_oct.csv',sep=';', names=['Date', 'ming'], parse_dates=['Date'], date_parser=mydateparser, low_memory=False)
#convert the data to time-series type, by taking index to be the time
dataset=dataset.sort_values(by=['Date'],ascending=True)
#Data visualization
dataset.set_index('Date')["ming"].plot(figsize=(16,6),legend=True,grid=True)
plt.legend(['Real Gas Price'])
plt.show()
#Data preprocesssing steps: 
#Step 1 : data re-sampling 
dataset=dataset.set_index('Date').resample('5 Min').mean()
dataset["ming"].plot(figsize=(16,6),legend=True,grid=True)
plt.legend(['Real Gas Price'])
plt.show()
#Step 2 : removing outliers 
## calculate summary statistics
data_mean, data_std = mean(dataset["ming"]), std(dataset["ming"])
# identify outliers
cut_off = data_std * 2
lower, upper = data_mean - cut_off, data_mean + cut_off
for index,row in dataset.iterrows():
  if row["ming"] < lower:
    row["ming"]=lower
  elif row["ming"] > upper:
    row["ming"]=upper
#remove outliers 
outliers_removed = [x for x in dataset["ming"] if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))
#Data visualization
dataset["ming"].plot(figsize=(16,6),legend=True,grid=True)
plt.legend(['Real Gas Price'])
plt.show()



In [ ]:
#loading data 
mydateparser = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
gethdata = pd.read_csv('geth.csv',sep=';', names=['Date', 'gethprice'], parse_dates=['Date'], date_parser=mydateparser, low_memory=False)
#convert the data to time-series type, by taking index to be the time
gethdata=gethdata.sort_values(by=['Date'],ascending=True)
#Data visualization
gethdata.set_index('Date')['2020-10-21 08:40:00':'2020-10-24 17:05:00']["gethprice"].plot(figsize=(16,6),legend=True,grid=True)
plt.legend(['Geth gas price'])
plt.show()
#Data preprocesssing steps: 
#Step 1 : data re-sampling 
gethdata=gethdata.set_index('Date')['2020-10-21 08:40:00':'2020-10-24 17:05:00'].resample('5 Min').mean()
#Step 2 : removing outliers 
# calculate summary statistics
data_mean, data_std = mean(gethdata["gethprice"]), std(gethdata["gethprice"])
# identify outliers
for index,row in gethdata.iterrows():
  if row["gethprice"] < lower:
    row["gethprice"]=lower
  elif row["gethprice"] > upper:
    row["gethprice"]=upper
#remove outliers 
outliers_removed = [x for x in gethdata["gethprice"] if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))
#Data visualization
gethdata['2020-10-21 08:40:00':'2020-10-24 17:05:00']["gethprice"].plot(figsize=(16,6),legend=True,grid=True)
plt.legend(['Geth gas price'])
plt.show()

Our approach proposes comparing different prediction models to recommend the gas price with Geth oracle.

# **Prophet model  :**
Prophet, developed by Facebook,  is an open-source tool designed for time series forecasting . It is implemented in an R library, and also a Python package (as already shown in this contest).

Prophet essentially is an additive regression model that decomposes a time series into (i) a linear/logistic (piecewise) trend, (ii) an annual seasonal component, (iii) a weekly seasonal component, and (iv) an optional list of important days (such as vacations, special events, ...). The model states that it is "robust to missing data, trend changes, and significant outliers", which would make it well suited to this particular task.




In [ ]:
#These experiments are done without data transformation technique 
dataset1 =dataset[['ming']]
#splitting data into Train/Test data
training_set = dataset1['2020-10-10 00:00:00':'2020-10-21 08:35:00']
training_set = training_set.reset_index()[['Date', 'ming']]
testsetp = dataset1['2020-10-21 08:40:00': '2020-10-24 18:05:00']
testsetp = testsetp.reset_index()[['Date', 'ming']]
training_set.columns = ['ds', 'y']
#Create Prophet Model
m = Prophet(changepoint_prior_scale=0.0005,yearly_seasonality=False, weekly_seasonality=True, daily_seasonality=True).fit(training_set)
#Forecast on Test Set
future = m.make_future_dataframe(periods=978, freq='5 Min', include_history= True)
fcst = m.predict(future)
#forecasts visualisation
fig = m.plot(fcst)
fig = m.plot_components(fcst)
real=np.array(dataset1.ming)
real=real.reshape(-1, 1)
pred=np.array(fcst.yhat)
pred=pred.reshape(-1, 1)
sc = MinMaxScaler(feature_range=(0,1))
real = sc.fit_transform(real)
pred = sc.transform(pred)
#Error Metrics On Test Set
print('prophet MAE : %.3f'%mean_absolute_error(real, pred))
print('prophet MSE: %.3f'%mean_squared_error(real, pred))
print('prophet R2-score : %.3f'%r2_score(real, pred))
print('prophet RMSE : %.3f'%sqrt(mean_squared_error(real, pred)))
fcst=fcst.set_index('ds')
dataset1=dataset1.reset_index()[['Date', 'ming']]
dataset1.columns = ['ds', 'y']
fcst = fcst.reset_index()[['ds', 'yhat']]

Brief notes about how the Prophet worked in practice:

data format: the prophet expects a data frame with only two columns: ds, y. The first column holds the dates, while the second contains the time series information.

The parameter changepoint.prior.scale is useful for adjusting the flexibility of the trend. Raising this parameter makes the adjustment more flexible, but it also increases the uncertainty of the forecast and makes it more likely to adapt to noise. The change points in the data are automatically detected, except if they are manually specified using the change point parameter (which we do not do here).

The parameters daily.seasonality=TRUE and weekly.seasonality=TRUE must be explicitly enabled and allow the prophet to notice small-scale cycles. 

In [ ]:
#These experiments are done with data transformation technique 
dataset1 =dataset[['ming']]
#applying logarithmic transformation
dataset1['ming'] = np.log(dataset1['ming'])
#splitting data into Train/Test data
training_set = dataset1['2020-10-10 00:00:00':'2020-10-21 08:35:00']
training_set = training_set.reset_index()[['Date', 'ming']]
testsetp = dataset1['2020-10-21 08:40:00': '2020-10-24 18:05:00']
testsetp = testsetp.reset_index()[['Date', 'ming']]
training_set.columns = ['ds', 'y']
#Create Prophet Model
m = Prophet(changepoint_prior_scale=0.005,yearly_seasonality=False, weekly_seasonality=True, daily_seasonality=True).fit(training_set)
#Forecast on Test Set
future = m.make_future_dataframe(periods=978, freq='5 Min', include_history= True)
fcst = m.predict(future)
#forecasts visualisation
fig = m.plot(fcst)
fig = m.plot_components(fcst)
#Error Metrics On Test Set
real=np.array(dataset1.ming)
real=real.reshape(-1, 1)
pred=np.array(fcst.yhat)
pred=pred.reshape(-1, 1)
sc = MinMaxScaler(feature_range=(0,1))
real = sc.fit_transform(real)
pred = sc.transform(pred)
print('prophet : %.3f'%mean_absolute_error(real, pred))
print('prophet : %.3f'%mean_squared_error(real, pred))
print('prophet : %.3f'%r2_score(real, pred))
print('prophet : %.3f'%sqrt(mean_squared_error(real, pred)))
fcst['yhat']=np.exp(fcst['yhat'])
dataset1['ming'] =np.exp(dataset1['ming'])
fcst=fcst.set_index('ds')
dataset1=dataset1.reset_index()[['Date', 'ming']]
dataset1.columns = ['ds', 'y']
fcst = fcst.reset_index()[['ds', 'yhat']]
#inverse the log transformation


In [ ]:
#visualize prophet prediction
metric_df = fcst.set_index('ds')[['yhat']].join(dataset1.set_index('ds').y).reset_index()
metric_df.y=metric_df.y.fillna(0)
print(metric_df.tail())
plt.legend(['true' , 'prdicted'])
metric_df.set_index('ds').y.plot(figsize=(16,6),legend=True,grid=True)
metric_df.set_index('ds').yhat.plot(figsize=(16,6),legend=True,grid=True)
plt.legend(['real gasprice' , 'predicted gas price'])
plt.title('Prophet Gas Price Predicton ')
plt.show()

In [ ]:
#visualize prophet prediction over training and testing  period 
plt.legend(['true' , 'predicted'])
metric_df.set_index('ds')['2020-10-10 00:00:00':'2020-10-21 08:35:00'].y.plot(figsize=(16,6),legend=True,grid=True)
metric_df.set_index('ds')['2020-10-10 00:00:00':'2020-10-21 08:35:00'].yhat.plot(figsize=(16,6),legend=True,grid=True)
plt.legend(['real gasprice' , 'predicted gas price'])
plt.title('Prophet Gas Price Predicton over training period')
plt.show()
plt.legend(['true' , 'prdicted'])
metric_df.set_index('ds')['2020-10-21 08:40:00': '2020-10-24 18:05:00'].y.plot(figsize=(16,6),legend=True,grid=True)
metric_df.set_index('ds')['2020-10-21 08:40:00': '2020-10-24 18:05:00'].yhat.plot(figsize=(16,6),legend=True,grid=True)
plt.legend(['real gasprice' , 'predicted gas price'])
plt.title('Prophet Gas Price Predicton over testing period')
plt.show()

GRU is a simplification of the LSTM unit, it has been demonstrated to work in a similar way in many tasks, while reducing complexity. GRU surpassed the LSTM in many cases. Therefore, if the data scale is large, LSTM may be more efficient. In the GRU, the input gate and the forgetting gate of LSTM have been merged into a single gate: the update gate. This unit controls the amount of information from the previous hidden state to the next hidden state, allowing long-term dependencies to be captured directly without the need for the cell state. To control the redundant information, there is a second gate, the reset gate that decides how much previous hidden states should be forgotten. The specificity of these gates is that they can be trained to keep information from a long time ago, without washing it through time or removing any not relevant information.

In [ ]:
dataset2 = dataset[['ming']]
# removing missing value rows
dataset2 = dataset2.dropna() 
#splitting data into Train/Test data
training_set = dataset2['2020-10-10 00:00:00':'2020-10-21 08:35:00']
test_set = dataset2['2020-10-21 07:20:00':'2020-10-24 17:05:00']
#applying min-max normalization
sc = MinMaxScaler(feature_range=(0,1))
training_set = sc.fit_transform(training_set)
test_set = sc.transform(test_set)
training_set = np.array(training_set)
test_set = np.array(test_set)
#Create x, y test and train data windows and into input and outputs
X_train = []
Y_train = []
previous = 15
# reshape input to be 2D [samples, timesteps]
for i in range(len(training_set)-previous-1):
    X_train.append(training_set[i:i+previous])
    Y_train.append(training_set[i+previous])
X_train, Y_train = np.array(X_train), np.array(Y_train)
Y_train = np.reshape(Y_train, (Y_train.shape[0],-1))
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_train.shape, Y_train.shape
X_test = []
Y_test = []
for i in range(len(test_set)-previous-1):
    X_test.append(test_set[i:i+previous])
    Y_test.append(test_set[i+previous])
X_test, Y_test = np.array(X_test), np.array(Y_test)
Y_test = np.reshape(Y_test, (Y_test.shape[0],-1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
#Create GRU Model
model = Sequential()
model.add(GRU(units=15,activation='tanh',input_shape=(X_train.shape[1], X_train.shape[2]),dropout=0.01))
model.add(Dense(1))
opt = SGD(lr=0.01, decay=0.0001)
model.compile(loss='mean_absolute_error', optimizer=opt)
# fit network
history = model.fit(X_train, Y_train, epochs=500,batch_size=128,shuffle=False,validation_split=0.10,verbose=0)
# summarize history for loss
train_maeG = modelG.evaluate(X_train,Y_train)
test_maeG = modelG.evaluate(X_test, Y_test)
print('Train: %.3f, Test: %.3f' % (train_maeG, test_maeG))
#visualize results 
plt.plot(history.epoch, history.history['loss'])
plt.plot(history.epoch , history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# make a prediction for training data
trainGRU = model.predict(X_train)
#calculate error rates 
maeGRUTR=mean_absolute_error(Y_train, trainGRU)
mseGRUTR=mean_squared_error(Y_train, trainGRU)
r2GRUTR=r2_score(Y_train, trainGRU)
rmseGRUTR=sqrt(mean_squared_error(Y_train, trainGRU))
print('gru : %.3f'%maeGRUTR)
print('gru : %.3f'%mseGRUTR)
print('gru : %.3f'%r2GRUTR)
print('gru : %.3f'%rmseGRUTR)
print(trainGRU.shape)
# invert scaling for forecast and real 
trainGRU = sc.inverse_transform(trainGRU)
Y_train = sc.inverse_transform(Y_train)
#visualaize forecastings 
plt.figure(figsize=(18,5))
plt.plot(Y_train, color='blue',label='Predicted '+' Gas Price')
plt.plot(trainGRU, color='yellow',label='real '+' Gas Price')
plt.title(' Gas Price Prediction(GRU)')
plt.xlabel('Time')
plt.grid()
plt.ylabel(' Gas Price')
plt.legend()
plt.show()

In [ ]:
testGRU = model.predict(X_test)
#calculate error rates 
maeGRUTS=mean_absolute_error(Y_test, testGRU)
mseGRUTS=mean_squared_error(Y_test, testGRU)
r2GRUTS=r2_score(Y_test, testGRU)
rmseGRUTS=sqrt(mean_squared_error(Y_test, testGRU))
print('gru : %.3f'%maeGRUTS)
print('gru : %.3f'%mseGRUTS)
print('gru : %.3f'%r2GRUTS)
print('gru : %.3f'%rmseGRUTS)
print(testGRU.shape)
# invert scaling for forecast and real
testGRU = sc.inverse_transform(testGRU)
Y_test = sc.inverse_transform(Y_test)
#visualaize forecastings
plt.figure(figsize=(18,5))
plt.plot(Y_test, color='blue',label='Predicted '+' Gas Price')
plt.plot(testGRU, color='yellow',label='real '+' Gas Price')
plt.title(' Gas Price Prediction(GRU)')
plt.xlabel('Time')
plt.grid()
plt.ylabel(' Gas Price')
plt.legend()
plt.show()

LSTM stands for long short term memory. It is a model or architecture that extends the memory of recurrent neural networks. Typically, recurrent neural networks have ‘short term memory’ in that they use persistent previous information to be used in the current neural network. Essentially, the previous information is used in the present task. That means we do not have a list of all of the previous information available for the neural node. LSTM introduces long-term memory into recurrent neural networks. It mitigates the vanishing gradient problem, which is where the neural network stops learning because the updates to the various weights within a given neural network become smaller and smaller. It does this by using a series of ‘gates’.These are contained in memory blocks which are connected through layers. LSTM work There are three types of gates within a unit: Input Gate: Scales input to cell (write) Output Gate: Scales output to cell (read) Forget Gate: Scales old cell value (reset) Each gate is like a switch that controls the read/write, thus incorporating the long-term memory function into the model.

In [ ]:
dataset2 = dataset[['ming']]
# removing missing value rows
dataset2 = dataset2.dropna()
#splitting data into Train/Test data
training_set = dataset2['2020-10-10 00:00:00':'2020-10-21 08:35:00']
test_set = dataset2['2020-10-21 07:20:00':'2020-10-24 17:05:00']
#applying min-max normalization 
sc = MinMaxScaler(feature_range=(0,1))
training_set = sc.fit_transform(training_set)
test_set = sc.transform(test_set)
#Create x, y test and train data windows and into input and outputs
training_set = np.array(training_set)
test_set = np.array(test_set)
x_train = []
y_train = []
previous =15
for i in range(len(training_set)-previous-1):
    x_train.append(training_set[i:i+previous])
    y_train.append(training_set[i+previous])
x_train, y_train = np.array(x_train), np.array(y_train)
# reshape input to be 3D [samples, timesteps]
y_train = np.reshape(y_train, (y_train.shape[0],-1))
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_train.shape, y_train.shape
x_test = []
y_test = []
for i in range(len(test_set)-previous-1):
    x_test.append(test_set[i:i+previous])
    y_test.append(test_set[i+previous])
x_test, y_test = np.array(x_test), np.array(y_test)
# reshape input to be 2D [samples, timesteps]
y_test = np.reshape(y_test, (y_test.shape[0],-1))
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
#Create LSTM Model
modelL = Sequential()
modelL.add(LSTM(units =15,activation='tanh',input_shape=(x_train.shape[1], x_train.shape[2]),dropout=0.01))
modelL.add(Dense(1))
opt = SGD(lr=0.01, decay=0.0001)
modelL.compile(loss='mean_absolute_error', optimizer=opt)
# fit network
history2 = modelL.fit(x_train, y_train, epochs=500,batch_size=112,validation_split=0.10,shuffle=False,verbose=0)
# summarize history for loss
train_mseL = modelL.evaluate(x_train,y_train)
test_mseL = modelL.evaluate(x_test, y_test)
print('Train: %.3f, Test: %.3f' % (train_mseL, test_mseL))
#visualize results 
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# make a prediction for training data
trainLSTM = modelL.predict(x_train)
#calculate error rates 
maeLSTMTR=mean_absolute_error(y_train, trainLSTM)
mseLSTMTR=mean_squared_error(y_train, trainLSTM)
r2LSTMTR=r2_score(y_train, trainLSTM)
rmseLSTMTR=sqrt(mean_squared_error(y_train, trainLSTM))
print('LSTM : %.3f'%maeLSTMTR)
print('LSTM : %.3f'%mseLSTMTR)
print('LSTM : %.3f'%r2LSTMTR)
print('LSTM : %.3f'%rmseLSTMTR)
print(trainLSTM.shape)
# invert scaling for forecast and real 
trainLSTM = sc.inverse_transform(trainLSTM)
y_train = sc.inverse_transform(y_train)
plt.figure(figsize=(18,5))
#visualaize forecastings 
plt.plot(y_train, color='blue',label='Predicted '+' Gas Price')
plt.plot(trainLSTM, color='red',label='real '+' Gas Price')
plt.title(' Gas Price Prediction(LSTM)')
plt.xlabel('Time')
plt.grid()
plt.ylabel(' Gas Price')
plt.legend()
plt.show()

In [ ]:
# make a prediction for training data
testLSTM = modelL.predict(x_test)
#calculate error rates 
maeLSTMTS=mean_absolute_error(y_test, testLSTM)
mseLSTMTS=mean_squared_error(y_test, testLSTM)
r2LSTMTS=r2_score(y_test, testLSTM)
rmseLSTMTS=sqrt(mean_squared_error(y_test, testLSTM))
print('LSTM : %.3f'%maeLSTMTS)
print('LSTM : %.3f'%mseLSTMTS)
print('LSTM : %.3f'%r2LSTMTS)
print('LSTM : %.3f'%rmseLSTMTS)
#calculate Geth error rates 
gethdata = gethdata.reset_index()[['Date', 'gethprice']]
geth=np.array(gethdata['gethprice'])
geth = sc.transform(geth.reshape(-1,1))
maeGeth=mean_absolute_error(y_test, geth)
print('Geth : %.3f'%mean_absolute_error(y_test, geth))
print('Geth : %.3f'%mean_squared_error(y_test, geth))
print('Geth : %.3f'%r2_score(y_test, geth))
print('Geth : %.3f'%sqrt(mean_squared_error(y_test, geth)))
# invert scaling for forecast and real 
testLSTM = sc.inverse_transform(testLSTM)
y_test = sc.inverse_transform(y_test)
#visualaize forecastings 
plt.figure(figsize=(18,5))
plt.plot(y_test, color='blue',label='Predicted '+' Gas Price')
plt.plot(testLSTM, color='red',label='real '+' Gas Price')
plt.title(' Gas Price Prediction(LSTM)')
plt.xlabel('Time')
plt.grid()
plt.ylabel(' Gas Price')
plt.legend()
plt.show()

In [ ]:
#visualize of different prediction models
gethdata['predP']=fcst['yhat'][0:966]
gethdata['predL']=testLSTM[0:966]
gethdata['predG']=testGRU[0:966]
gethdata['real']=y_test[0:966]
gethdata.set_index('Date').gethprice.plot(figsize=(18,6),color='g',legend=True,grid=True)
gethdata.set_index('Date').real.plot(figsize=(18,6),color='b',legend=True,grid=True)
gethdata.set_index('Date').predL.plot(figsize=(18,6),color='r',legend=True,grid=True)
gethdata.set_index('Date').predG.plot(figsize=(18,6),color='y',legend=True,grid=True)
gethdata.set_index('Date').predP.plot(figsize=(18,6),color='orange',legend=True,grid=True)
plt.legend(['Geth' , 'real','predLSTM','predGRU','predProphet'])
plt.title(' Gas Price methods comparison')
plt.show()